# Preparar el paquete oficial sin subir binarios
Este cuaderno automatiza la preparación de la carpeta `files/`
utilizando el paquete original del profesor (`Files-20251006.zip`).
Puedes mantener el ZIP fuera de Git y ejecutar estas celdas
cada vez que necesites reconstruir las imágenes de práctica.

1. Descarga `Files-20251006.zip` de la plataforma oficial.
2. Copia el archivo dentro de `Files-20250930 (2)/`.
3. Ejecuta la celda siguiente para regenerar `files/images/` y la carpeta
   de salida `files/reduced_images/`.

Si ya tienes `files/images/` en tu copia de trabajo no es necesario volver
a ejecutar nada: el cuaderno detecta automáticamente ese caso.

In [ ]:
from pathlib import Path
import tarfile
import zipfile

DATA_DIR = Path('Files-20250930 (2)')
FILES_DIR = DATA_DIR / 'files'
IMAGES_DIR = FILES_DIR / 'images'
OUTPUT_DIR = FILES_DIR / 'reduced_images'
ZIP_ARCHIVE = DATA_DIR / 'Files-20251006.zip'
IMAGES_ARCHIVE = FILES_DIR / 'images.tar'

def _extract_tar() -> None:
    with tarfile.open(IMAGES_ARCHIVE, mode='r') as archive:
        archive.extractall(path=FILES_DIR)
    print(f'Extraído {IMAGES_ARCHIVE} → {IMAGES_DIR}')

def _extract_zip() -> None:
    with zipfile.ZipFile(ZIP_ARCHIVE) as bundle:
        bundle.extractall(path=DATA_DIR)
    print(f'Descomprimido {ZIP_ARCHIVE} → {FILES_DIR}')

def prepare_official_files() -> Path:
    FILES_DIR.mkdir(parents=True, exist_ok=True)
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    if IMAGES_DIR.exists() and any(IMAGES_DIR.iterdir()):
        print('Las imágenes ya están disponibles:',
              ', '.join(sorted(p.name for p in IMAGES_DIR.iterdir())))
        return IMAGES_DIR

    if IMAGES_ARCHIVE.exists():
        _extract_tar()
    elif ZIP_ARCHIVE.exists():
        _extract_zip()
        if IMAGES_ARCHIVE.exists():
            _extract_tar()
        else:
            raise FileNotFoundError(
                'El ZIP se ha descomprimido pero no contiene images.tar. '
                'Revisa que sea el paquete oficial.'
            )
    else:
        raise FileNotFoundError(
            'No se encontró ni images.tar ni Files-20251006.zip. '
            'Descarga el ZIP del campus virtual y colócalo junto a este cuaderno.'
        )

    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    if not any(IMAGES_DIR.iterdir()):
        raise RuntimeError('No se pudieron extraer las imágenes de práctica.')

    print('Preparación completada. Carpeta de salida lista en', OUTPUT_DIR)
    return IMAGES_DIR

prepare_official_files()


Puedes verificar la integridad del ZIP oficial ejecutando la celda
siguiente. Si todo va bien Python devolverá `None`.

In [ ]:
if ZIP_ARCHIVE.exists():
    result = zipfile.ZipFile(ZIP_ARCHIVE).testzip()
    print('Resultado del test:', result)
else:
    print('Coloca Files-20251006.zip para ejecutar esta comprobación.')
